## Get frames With a person (or more on it)

This script aims to retrieve people frames from different cameras to generate datasets.

In [ ]:
from datetime import datetime, timedelta, timezone
import random
import sys
from collections import defaultdict
from tqdm import tqdm
from glob import glob
import os

sys.path.append('..')
from event_query_app.event_query import EventQuery

DATASET_DST = "/home/gfuhr/projects/data/immediate_action/people_v1/"
if not os.path.exists(DATASET_DST):
    os.makedirs(DATASET_DST)

CLASS = "person"
DEVICE_IDS = [
    "B8A44F976508__panoramic_tree",
    "B8A44F9C9902__train_rails",
    "B8A44FB97C3C__panoramic_garages",
    "B8A44FCDF536__parking_lot",
    "B8A44FD014E5__distorted_tree",
    "B8A44FE820CC__construction_ptz",
    "B8A44F9CA507__front_far",
    "B8A44FB0AC74__trains_close",
    "B8A44FB3A1F9__front",
    "B8A44FB981BB__panoramic_trains",
    "B8A44FE6C853__booth_exit",
    "B8A44FE6D078__booth_entrance"
]

THRESHOLD = 0.7
CAP_PER_DAY_PER_DEVICE = 20
DAYS_BACK = 30
FRAMES_PER_EVENT = 3

event_query = EventQuery()
END_DATE = datetime.now(timezone.utc)

from kvs_event_consumer import KVSEventConsumer
import numpy as np
import cv2

kvs_consumer = KVSEventConsumer()
for device_id in DEVICE_IDS:
    for day_offset in range(DAYS_BACK):
        day_start = END_DATE - timedelta(days=day_offset + 1)
        day_end = END_DATE - timedelta(days=day_offset)
        print(f"Processing events from {day_start} to {day_end} for device {device_id}")

        daily_results = event_query.query_events(
            start_date=day_start,
            end_date=day_end,
            target_classes=[CLASS],
            threshold=THRESHOLD,
            device_ids=[device_id]
        )

        sampled_daily_events = random.sample(daily_results, min(CAP_PER_DAY_PER_DEVICE, len(daily_results)))
        for event in tqdm(sampled_daily_events):
            
            existing_files = glob(os.path.join(DATASET_DST, f"{device_id}_event_{event['event_timestamp']}_frame_*.png"))
            if existing_files:
                continue  # Skip if already processed

            frames = kvs_consumer.get_event_frames(event)
            indices = np.linspace(0, len(frames) - 1, num=3, dtype=int)
            for i in indices:
                frame_filepath = os.path.join(DATASET_DST, f"{device_id}_event_{event['event_timestamp']}_frame_{i}.png")
                frame_bgr = cv2.cvtColor(frames[i], cv2.COLOR_RGB2BGR)
                cv2.imwrite(frame_filepath, frame_bgr)
            
        